In [8]:

from flask import Flask, jsonify
from flask_restx import Api, Resource, fields, reqparse
import pickle
from watchdog.events import EVENT_TYPE_CREATED, EVENT_TYPE_OPENED
import pandas as pd
from tqdm import tqdm
import re


In [9]:
app = Flask(__name__)
api = Api(app, version='1.0', title='Modelo de Clasificación de Peliculas',
          description='API para clasificar peliculas según su trama o plot')


In [12]:
# Procesamiento de los datos de Entrada
#Importación del modelo a utilizar 
model = pickle.load(open('modelo__clasificacion_entrenado.pkl', 'rb'))
vect = pickle.load(open('vect.pkl', 'rb'))
tfidf = pickle.load(open('tfidf_transformer.pkl', 'rb'))


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# Creación del namespace para los endpoints
namespace = api.namespace('predict', description='Clasificación de peliculas)

parser = api.parser()
parser.add_argument('plot', type=str, required=True, help='Plot o Trama de la pelicula', location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})


@api.route('/')
class Predict(Resource):
    @api.doc(parser=parser)
    def get(self):
        # Parseamos los parámetros de entrada
        args = parser.parse_args()
        print(f'Argumentos recibidos: {args}')
        
        # Creamos una matriz con los valores de entrada
        input_data = [[args['plot']]
        df = pd.DataFrame(input_data, columns=['Plot'])
        print(f'DataFrame generado: \n{df}')
        
      
        preprocessed_plot = []

        for sentance in df['plot'].values:
        sentance = decontracted(sentance)
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed_plot.append(sentance.strip())

        df['preprocessed_plot']=preprocessed_plot
        
                      
        X_test_dtm_1 = vect.transform(df['preprocessed_plot'])
        X_test_dtm = tfidf.transform(X_test_dtm_1)

        # Transformamos los valores de entrada utilizando el preprocesador
        print(f'DataFrame transformado: \n{X_test_dtm}')
        
        
        # Realizamos la predicción utilizando el modelo entrenado
        predicted_plot = model.predict(X_test_dtm)
        print(f'Precio predicho: {predicted_plot}')
        
        
        # Devolvemos la predicción como un diccionario JSON
        return jsonify({'predicted_price': predicted_plot})
        
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

SyntaxError: EOL while scanning string literal (Temp/ipykernel_16100/1232381448.py, line 26)

In [ ]:

# Cargar los modelos y objetos necesarios
model = pickle.load(open('modelo__clasificacion_entrenado.pkl', 'rb'))
vect = pickle.load(open('vect.pkl', 'rb'))
tfidf = pickle.load(open('tfidf_transformer.pkl', 'rb'))

# Función de expansión de contracciones
def decontracted(phrase):
    # Específicas
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)

    # Generales
    phrase = re.sub(r"n't", " not", phrase)
    phrase = re.sub(r"'re", " are", phrase)
    phrase = re.sub(r"'s", " is", phrase)
    phrase = re.sub(r"'d", " would", phrase)
    phrase = re.sub(r"'ll", " will", phrase)
    phrase = re.sub(r"'t", " not", phrase)
    phrase = re.sub(r"'ve", " have", phrase)
    phrase = re.sub(r"'m", " am", phrase)
    return phrase


# Crear la aplicación Flask
app = Flask(__name__)
api = Api(app)

# Definir el namespace y el parser para los endpoints
namespace = api.namespace('predict', description='Clasificación de películas')

parser = api.parser()
parser.add_argument('plot', type=str, required=True, help='Trama de la película', location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})


@api.route('/')
class Predict(Resource):
    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        # Obtener los parámetros de entrada
        args = parser.parse_args()
        plot = args['plot']
        
        # Preprocesar los datos de entrada
        preprocessed_plot = decontracted(plot)
        preprocessed_plot = re.sub('[^A-Za-z]+', ' ', preprocessed_plot)
        preprocessed_plot = ' '.join(e.lower() for e in preprocessed_plot.split() if e.lower() not in stopwords)
        
        # Crear el DataFrame con los datos preprocesados
        df = pd.DataFrame({'Plot': [preprocessed_plot]})
        
        # Transformar los datos de entrada utilizando el vectorizador y el transformador TF-IDF
        X_test_dtm_1 = vect.transform(df['Plot'])
        X_test_dtm = tfidf.transform(X_test_dtm_1)
        
        # Realizar la predicción utilizando el modelo entrenado
        predicted_genre = model.predict(X_test_dtm)
        
        # Devolver la predicción como un diccionario JSON
        return {'result': predicted_genre[0]}


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://192.168.1.3:5000/ (Press CTRL+C to quit)
192.168.1.3 - - [27/May/2023 23:18:31] "GET / HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:18:33] "GET /swagger.json HTTP/1.1" 200 -
192.168.1.3 - - [27/May/2023 23:19:05] "GET /?plot=One%20day%20in%20the%20city%20 HTTP/1.1" 200 -
